## 关联分析的定义

关联是反映某个事物与其他事物之间的相互依存关系，关联分析是指在交易数据中，找出存在于项目集合之间的关联模式，即两个或者多个事物之间存在一定的关联性，则可以通过一个事物预测其他事物。换句话说，两项或者多项属性之间存在关联，就可以用其中一项的属性值预测另一项或另几项。例如描述牛奶和面包的关联规则（买面包的人也会买牛奶）：买面包-->买牛奶。关联关系的数学表达：
$$X \rightarrow Y, X \cap Y = \emptyset$$

## 概念
### 基本概念
|订单编号 | 购买商品|
| --- | --- |
| 1 | 牛奶、面包、尿布 |
| 2 | 可乐、面包、尿布、啤酒 |
| 3 | 牛奶、尿布、啤酒、鸡蛋 |
| 4 | 面包、牛奶、尿布、啤酒 |
| 5 | 面包、牛奶、尿布、可乐 |

- 事务：每条交易称为一个事务，上表就包含5个事务。
- 项：交易的每个物品称为一个项，如面包、牛奶等。
- 项集：包含零个或者多个项的集合叫做项集，例如{牛奶、面包}。
- 规则：从众多项集中找出的各项之间的关系，如关联规则${牛奶} \rightarrow {面包}$

### 关联关系有效性指标
- 支持度（support）
- 
指某数据集中包含某几个特定项的事务出现的概率。例如，上面的表格中，$support(牛奶)=4/5$，$support({牛奶、面包})=3/5$。支持度反映了项集或规则的普遍程度。
$$s_{X-Y} = \frac{N(XY)}{N}$$
其中$N(XY)$为同事购买X和Y的事务的数量，$N$为数据集总事务量。

- 置信度（confidence）

在给定前项X的前提下，后项Y发生的概率，是一个条件概率。例如，$confidence(买牛奶 \rightarrow 买面包)= 3/4$.反映了规则的可靠度。
$$c_{X \rightarrow Y} = \frac{N(XY)}{N(X)}$$

- 提升度（lift）

先购买某一商品对购买另一商品的提升作用。衡量了购买某一商品X时购买另一商品Y比单独购买商品Y的概率的提升程度。例如，$lift(买牛奶 \rightarrow 买面包) = c(买牛奶 \rightarrow 买面包)/s(买面包)$。用于判断商品组合方式是否具有实际价值。若$lift > 1$则表明有提升，组合有价值；若$lift = 1$则表明没有提升也没有下降；若$lift < 1$则表明有下降，组合无价值。
$$l_{X \rightarrow Y} = \frac{c_{X \rightarrow Y}}{s_{Y}}$$

- 频繁项集

若某一个项集的支持度大于预设的最小支持度（minsupport），则称这个项集为频繁项集。频繁项集也就是关联分析中所要找到的目标。

<font color="red">那么如何寻找频繁项集呢？当然我们可以根据上述概念描述的朴素方法寻找，但是数据量过大显然这个样子是不合适的。下面就介绍两个算法来实现寻找频繁项集。</font>


## 算法实现
### apriori算法
核心思想：如果某个项集是频繁项集，那么它的子集也是频繁项集。逆否命题为：如果某个项集的子集不是频繁项集，那么这个项集也不是频繁项集。

证明：
$$s({A、B}) = \frac{N(AB)}{N} \leq \frac{N(ABC)}{N} = s({A、B、C}) \leq minsupport$$
同理可得{A、B、C}的其他子集也满足上式。


主要步骤：

1. 根据数据集生成候选项，首先生成单项集。

2. 设定最小支持度和最小置信度。

3. 过滤掉数据项集中占比低于最小支持度的单项集中的项，只留下为频繁项集的单项集的项。

4. 根据步骤3形成的新的数据集，进行项集之间的组合形成新的项集集合，再寻找频繁二项集。

5. 重复步骤3、4，逐级寻找频繁三项集、四项集等，直到没有新的项集满足最小支持度。

6. 根据步骤5形成的最终频繁项集，生成关联规则。

7. 根据关联规则计算置信度，过滤掉小于最小置信度的关联规则。

### apriori算法示例

#### 表格例子

最小支持度设定为 0.6

| TID | Items   |
| --- | ------- |
| T1  | A, B, C |
| T2  | A, C    |
| T3  | B, C    |
| T4  | A, B    |
| T5  | A, B, C |


---

| Frequent 1-Itemset | Support |
| ------------------ | ------- |
| {A}                | **0.8** |
| {B}                | **0.8** |
| {C}                | **0.8** |

频繁单项集：{A}{B}{C}

---

| Frequent 2-Itemset | Support |
| ------------------ | ------- |
| {A, B}             | **0.6** |
| {A, C}             | **0.6** |
| {B, C}             | **0.6** |

频繁二项集：{A、B}{A、C}{B、C}

---

| Itemset   | Count | Support       |
| --------- | ----- | ------------- |
| {A, B, C} | 2     | 2/5 = **0.4** |

无频繁三项集

---
频繁项集

| Level | Frequent Itemsets   | Support（概率）   |
| ----- | ------------------- | ------------- |
| L1    | {A}, {B}, {C}       | 0.8, 0.8, 0.8 |
| L2    | {A,B}, {A,C}, {B,C} | 0.6, 0.6, 0.6 |



#### 不调用库

In [ ]:
def loadDataSet():
    # 定义一个示例数据集：每个子列表表示一笔事务（购买/出现的项集合）
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]
    # 说明：这是用来测试算法的小数据集，实际使用时替换为真实事务列表


def createC1(dataSet):
    # 从数据集中生成 C1（所有单项的候选集合，形式为列表的列表）
    C1 = []                       # 初始化空列表，用来存放单个项的列表形式（例如 [1], [2]）
    for transaction in dataSet:   # 遍历每一笔事务
        for item in transaction:  # 遍历事务中的每一个项
            if not [item] in C1:  # 如果 [item]（单元素列表）还没有被加入 C1，则加入
                C1.append([item]) # 避免重复加入同一单项
    # 此时 C1 是一个类似 [[1], [3], [4], [2], [5]] 的列表（顺序可能不同）
    C1.sort()                     # 对单项列表进行排序，便于生成候选集时保持稳定顺序
    return list(map(frozenset, C1))
    # 将每个单项列表转换为 frozenset（不可变集合），以便后续可以作为字典的键或集合操作
    # map 返回迭代器，list() 转回列表。最终返回类似 [frozenset({1}), frozenset({2}), ...]


def scanD(D, Ck, minSupport):
    """
    扫描数据库 D，统计候选集 Ck 的支持度，返回满足最小支持度的频繁项集列表（retList）和支持度字典（supportData）
    参数：
        D: 事务数据库，通常为 list of set 或 list of frozenset（这里假设已转换为集合形式）
        Ck: 候选项集列表（元素为 frozenset），需要统计它们在 D 中的出现次数
        minSupport: 最小支持度阈值（这里代码使用相对支持度，例如 0.5 表示 50%）
    """
    ssCnt = {}            # 用来计数每个候选项集的出现次数（字典：项集 -> 次数）
    for tid in D:         # 遍历数据库中的每一笔事务（每笔事务应为 set/frozenset）
        for can in Ck:    # 遍历每个候选项集
            if can.issubset(tid):  # 如果候选项集 can 是事务 tid 的子集（即该事务包含该候选项）
                if not can in ssCnt:
                    ssCnt[can] = 1  # 第一次计数则初始化为 1
                else:
                    ssCnt[can] += 1 # 否则在已有计数上加 1

    numItems = float(len(D))    # 数据集中事务的总数（转换成 float，方便后面做除法得到相对支持度）
    retList = []                # 用来保存满足最小支持度的频繁项集（以 frozenset 形式）
    supportData = {}            # 保存所有候选项集的支持度值（项集 -> 支持度）

    for key in ssCnt:           # 遍历所有被计数的候选项集
        support = ssCnt[key] / numItems  # 计算相对支持度 = 出现次数 / 总事务数
        if support >= minSupport:
            retList.insert(0, key)       # 如果满足最小支持度，将其插入 retList（insert(0,...) 在列表头部插入）
            # 注意：插入头部只是控制顺序，没有本质影响
        supportData[key] = support      # 将该候选项集的支持度记录到 supportData

    return retList, supportData     # 返回频繁项集列表和支持度字典


In [ ]:
def aprioriGen(Lk, k):  # creates Ck —— 用 L(k-1) 生成候选项集 Ck
    retList = []        # 用来存储生成的候选项集 Ck
    lenLk = len(Lk)     # 当前频繁项集 L(k-1) 的数量

    # 两两组合 L(k-1) 中的项集，尝试生成 k 项集
    for i in range(lenLk):
        for j in range(i + 1, lenLk):  # 避免重复组合
            L1 = list(Lk[i])[:k-2]     # 取第 i 个项集的前 k-2 个元素
            L2 = list(Lk[j])[:k-2]     # 取第 j 个项集的前 k-2 个元素

            L1.sort()                  # 排序以保证可比较
            L2.sort()

            # Apriori 剪枝：如果两个项集前 k-2 项相同，就可以合并生成 k 项集
            if L1 == L2:               # 若前 k-2 个元素完全一样
                retList.append(Lk[i] | Lk[j])  # 合并成 k 项集，使用集合的并操作 union

    return retList                     # 返回生成的候选 k 项集 Ck


#### 调用库

In [3]:
%pip install efficient-apriori

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [4]:
from efficient_apriori import apriori

In [5]:
dataset = [
    ['牛奶', '面包', '尿布'],
    ['可乐', '面包', '尿布'],
    ['牛奶', '尿布'],
    ['牛奶', '面包'],
    ['面包', '尿布'],
    ['牛奶', '可乐', '面包', '尿布']
]

itemsets, rules = apriori(dataset, min_support=0.5, min_confidence=0.7)

In [6]:
print(itemsets)
print(rules)

{1: {('牛奶',): 4, ('面包',): 5, ('尿布',): 5}, 2: {('尿布', '牛奶'): 3, ('尿布', '面包'): 4, ('牛奶', '面包'): 3}}
[{牛奶} -> {尿布}, {面包} -> {尿布}, {尿布} -> {面包}, {牛奶} -> {面包}]


### apriori算法优缺点
- 优点：容易上手
- 缺点：面对大数据时运行速度太慢


针对缺点再介绍一个新的算法 FP-growth算法（也叫FP树）


### FP-growth算法

核心思想：FP-growth主要采用一种分治的策略来解决该问题，我们可以用几个步骤来描述一下这种分治策略的大概步骤。

主要步骤：

1. 压缩数据集来表征每一个项，这个步骤一般是通过建立频繁模式树(frequent pattern tree，简称FP-tree)来实现的（其实就是字典树，很明显这是一种无损压缩方式）

2. 统计每一个项在原数据集中出现的次数，并根据预先设定的support count去除低频项，然后根据出现次数对剩余项进行升序排序；从第一位项开始扫描频繁模式树的叶子节点，通过回溯得到每一个项对应的条件模式基(Condition Pattern Base，实则为叶子节点对应的路径集合)

3. 根据条件模式基构建出条件频繁项集树(Conditional FP-tree，步骤和第一步完全一样，也就是根据条件模式基构建出一颗字典树)

4. 根据条件FP-tree和support count得到最终的频繁项集

<font color="red">注意：对于FP-growth算法设定的是 support count是数量而不是概率 </font>

### FP-growth算法实现
比较复杂，有时间了再研究

[点击参考知乎文章](https://zhuanlan.zhihu.com/p/411594391)

[FP-growth树求解过程视频版](https://www.bilibili.com/video/BV1fX4y1A75f/?spm_id_from=333.337.search-card.all.click&vd_source=62a21f026fe60108faad8418b1f5996d)

## 关联分析应用场景

1. 电商推荐与购物篮分析。电商数据是大量的交易项集（订单多事一揽子商品），这样的数据就非常适合做关联分析。用找到的关联规则进行则和套餐推荐、关联商品促销、超市陈列优化等促进销售，从而提升经营利润。

2. 内容推荐：看了某个或某类视频/文章的人还会看什么，进行关联推荐

3. 金融风控：某些异常行为与诈骗之间的关联规则，有助于发现异常行为的组合模式，风险指标之间的关联。

4. 用户反馈信息：在用户反馈信息中发现用户反映的问题之间的关联规则。

5. 游戏运营分析：玩家行为分析，打副本的玩家也会强化武器和购买药品，有助于理解玩家行为模式，进行游戏商城推荐。

## 关联分析优缺点


### 一、关联分析的优点

#### 1. 能挖掘潜在的关联关系，发现业务机会

关联分析能自动发现用户行为或商品之间的共现关系，是寻找隐藏模式最有效的方法之一。

例子：

* 买「牛奶」的用户经常买「面包」
* 跳到「支付失败」页面的用户更可能触发「联系客服」

它适用于：推荐、商品组合、运营策略、功能共现分析等。

---

#### 2. 可解释性强，业务易理解、可直接使用

Apriori/FP-growth 的结果中有明确的：

* 支持度（出现频率）
* 置信度（在 A 的情况下出现 B 的概率）
* 提升度（A→B 是否比随机更相关）

业务方能直观看懂结果，因此更容易落地。

---

#### 3. 可以基于简单事件日志快速构建

不需要复杂建模，只要有行为序列或交易数据就能运行。
适用于：

* 行为事件流数据
* 订单记录
* 功能点击序列
* 用户路径数据

门槛低，速度快。

---

#### 4. 支持多种业务应用场景

例如：

* 商品组合推荐（购物篮分析）
* 功能共用关系挖掘
* 用户群体行为模式识别
* 运营活动组合优化
* 流失路径中的高频共现事件识别

业务应用面非常广。

---

#### 5. 易于扩展与集成

关联规则可直接用于：

* 推荐系统候选集生成
* 用户细分规则构造
* 营销自动化策略（例如“做了 A 的用户推送 B”）

实用性高。

---

### 二、关联分析的缺点

#### 1. 容易产生大量无意义规则，筛选成本高

未加筛选时，可能会产生成百上千条规则，其中大部分：

* 业务不相关
* 符合直觉、没有价值
* 是由共性行为引起的虚假关联

需要人工筛选、设定阈值或使用提升度过滤。

---

#### 2. 无法识别“因果关系”，只有“共现关系”

关联分析只告诉你**A 和 B 经常一起发生**，但不会告诉你：

* 是 A 导致 B
* 是 B 导致 A
* 还是 C 导致 A 和 B 同时发生

例子：
“买尿布的人也买啤酒”并不代表尿布导致啤酒销量上升。

---

#### 3. 对稀疏数据不友好

如果用户行为很分散，关联模式就会极其稀疏，可能出现：

* 支持度很低
* 规则稀少或不稳定
* 即使有规则也不具备推广意义

典型场景：大量功能、页面、商品品类。

---

#### 4. 参数敏感：阈值（支持度/置信度）设定影响巨大

* 支持度设高 → 规则少，但可能漏掉有价值的模式
* 支持度设低 → 规则爆炸，噪音多
* 置信度/提升度适配不同业务含义

调参往往靠经验和业务理解，缺乏标准答案。

---

#### 5. 难以处理复杂行为序列（顺序、时间依赖）

标准关联分析只关注“是否一起出现”，忽略：

* 行为顺序（A 后 B 是否更重要？）
* 时间窗长短（5 分钟 vs 7 天）
* 行为强度（频次高是否更重要？）

对于时序关系，需要使用：

* 序列模式挖掘（PrefixSpan）
* Markov 链
* 因果推断
* 路径分析（Path Analysis）

单纯关联分析的表达能力有限。

---

#### 总结：一句话理解关联分析

> **关联分析擅长发现“是什么和什么一起发生”，但不擅长解释“为什么发生”。它的结果简单易懂，但需要结合业务筛选与解释，才能产生实际价值。**


